In [1]:
import nltk 
import discord
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle
import csv
import re
import pandas as pd
import pymongo
import pprint
import string
import numpy as np
import os

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/averygoldberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2022-12-15 16:08:01.345725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open("intents_netflix.json") as file:
    data = json.load(file)

try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)



net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)

Training Step: 14999  | total loss: 0.01990 | time: 0.094s
| Adam | epoch: 1000 | loss: 0.01990 - acc: 0.9802 -- iter: 112/114
Training Step: 15000  | total loss: 0.01795 | time: 0.100s
| Adam | epoch: 1000 | loss: 0.01795 - acc: 0.9821 -- iter: 114/114
--
INFO:tensorflow:/Users/averygoldberg/Documents/DS 2002/netflix/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /Users/averygoldberg/Documents/DS 2002/netflix/model.tflearn


In [7]:
def chat():
    host_name = "localhost"
    port = "27017"
    conn_str = {
    "local" : f"mongodb://{host_name}:{port}/",
    }
    client = pymongo.MongoClient(conn_str["local"])
    db_name = "netflix"
    netflix = client[db_name]
    print("Start talking with the bot! (type quit to stop)\n")
    print("Type \"help\" for information on what I can answer!\n")
    print("When asking questions, specify whether you are asking about movie or tv show data!\n")
    print('For reference, listed movie genres are:\ndocumentary\nscifi\ndrama\ncomedy\nwestern\naction\ncrime\nthriller\nwar\nfantasy\nromance\nhorror\nmusical\nanimation\nsports')
    while True:
        inp = input("You: ")
        inp = inp.translate(str.maketrans('', '', string.punctuation))
        if inp.lower() == "quit":
            break
        if inp.lower() == "help":
            print("Here are the types of questions I can answer:\n1. Finding the top show in a given year\n2. Finding the top movie in a given year\n3. Finding the top movie in a given genre\n4. Finding the top show in a given genre\n5. Finding the top 5 actors in the top show in a given year\n6. Finding the top 5 actors in the top movie in a given year\n7. Finding the number of seasons in the top show in a given year\n8. Finding the top 5 shows released in a given year\n9. Finding the top 5 movies released in a given year\n10. Finding the rating of the top movie in a given year")

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]
        inp = inp.split(' ')
        if "help" not in inp and "show" not in inp and "movie" not in inp and "shows" not in inp and "movies" not in inp and result[result_index] > 0.7:
            print("I didnt get that. Can you explain or try again?")
            print("Here are the types of questions I can answer:\n1. Finding the top show in a given year\n2. Finding the top movie in a given year\n3. Finding the top movie in a given genre\n4. Finding the top show in a given genre\n5. Finding the top 5 actors in the top show in a given year\n6. Finding the top 5 actors in the top movie in a given year\n7. Finding the number of seasons in the top show in a given year\n8. Finding the top 5 shows released in a given year\n9. Finding the top 5 movies released in a given year\n10. Finding the rating of the top movie in a given year")
            False
        if result[result_index] > 0.7:
            for tg in data["intents"]:
                key = None
                if tg['tag'] == 'q1': # Top show in [year]?
                    if "show" in inp and "seasons" not in inp and "season" not in inp and "Who" not in inp and "who" not in inp:
                        shows = []
                        genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                        for each in genres:
                            if each in inp:
                                False
                                break
                            else:
                                continue
                        for each in netflix.netflix.find( {"type": "SHOW"} ):
                            try:
                                shows.append(each["TITLE"])
                            except KeyError:
                                continue
                        for each in inp:
                            match = re.search(r'\d\d\d\d', each)
                            if match:
                                key = int(each)
                                show_names = []
                                shows = []
                                scores = []
                                options = []
                                for each in netflix.netflix.find( {"type": "SHOW"} ):
                                    show_names.append(each["title"])
                                for each in netflix.netflix.find():
                                    try:
                                        if each["TITLE"] in show_names:
                                            shows.append(each)
                                        for each in shows:
                                            if each["RELEASE_YEAR"] == key:
                                                scores.append(each["SCORE"])
                                            options.append(each)
                                        try:
                                            top = max(scores)
                                        except ValueError:
                                            continue
                                        for each in options:
                                            if ("SCORE", top) in each.items() and each["RELEASE_YEAR"] == key:
                                                title = each["TITLE"]
                                    except KeyError:
                                        break

                                print("The top show in",key, "was", title+".")
                            
                elif tg['tag'] == 'q2' and "show" not in inp and "shows" not in inp: # Top movie in [year]?
                    if "movie" in inp and "seasons" not in inp and "season" not in inp and "Who" not in inp and "who" not in inp:
                        shows = []
                        for each in netflix.netflix.find( {"type": "MOVIE"} ):
                            try:
                                shows.append(each["TITLE"])
                            except KeyError:
                                continue
                        for each in inp:
                            match = re.search(r'\d\d\d\d', each)
                            if match:
                                key = int(each)
                                show_names = []
                                shows = []
                                scores = []
                                options = []
                        for each in netflix.netflix.find( {"type": "MOVIE"} ):
                            show_names.append(each["title"])
                        for each in netflix.netflix.find():
                            try:
                                if each["TITLE"] in show_names:
                                    shows.append(each)
                                for each in shows:
                                    if each["RELEASE_YEAR"] == key:
                                        scores.append(each["SCORE"])
                                        options.append(each)
                                try:
                                    top = max(scores)
                                except ValueError:
                                    continue
                                for each in options:
                                    if ("SCORE", top) in each.items() and each["RELEASE_YEAR"] == key:
                                        title = each["TITLE"]
                            except KeyError:
                                break
                    try:
                        print("The top movie in",key, "was", title+".")
                    except UnboundLocalError:
                        continue

                elif tg["tag"] == "q3": # Top movie in [genre]?
                    if "movie" in inp  and "Who" not in inp and "who" not in inp:
                        try:
                            genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                            scores = []
                            movie_names = []
                            movies = []
                            for each in inp:
                                if each in genres:
                                    genre = each
                            for each in netflix.netflix.find({"type":"MOVIE"}):
                                movie_names.append(each["title"])

                            for items in netflix.netflix.find( {"MAIN_GENRE":genre}):
                                for each in movie_names:
                                    if ("TITLE", each) in items.items():
                                        scores.append(items["SCORE"])
                                        movies.append(items)
                                #if ("MAIN_GENRE",genre) in each.items():
                                    #scores.append(each['SCORE'])
                            top = max(scores)
                            for each in movies:
                                if ("SCORE",top) in each.items():
                                    title = each['TITLE']
                            print('The best',genre,"movie on Netflix is",title)
                        except UnboundLocalError:
                            continue
                elif tg["tag"] == "q4": # Top tv show in [genre]?
                    if "show" in inp  and "Who" not in inp and "who" not in inp:
                        try:
                            genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                            scores = []
                            show_names = []
                            shows = []
                            for each in inp:
                                if each in genres:
                                    genre = each
                            for each in netflix.netflix.find({"type":"SHOW"}):
                                show_names.append(each["title"])

                            for items in netflix.netflix.find( {"MAIN_GENRE":genre}):
                                for each in show_names:
                                    if ("TITLE", each) in items.items():
                                        scores.append(items["SCORE"])
                                        shows.append(items)
                                #if ("MAIN_GENRE",genre) in each.items():
                                    #scores.append(each['SCORE'])
                            top = max(scores)
                            for each in shows:
                                if ("SCORE",top) in each.items():
                                    title = each['TITLE']
                            print('The best',genre,"tv show on Netflix is",title)
                        except UnboundLocalError:
                            continue
                elif tg["tag"] == "q5": # Top 5 actors in top movie in [year]?
                    if "movie" in inp and "who" in inp:
                        print('Let me think...')
                        shows = []
                        actor = []
                        show_names = []
                        scores = []
                        options = []

                        for each in netflix.netflix.find( {"type": "MOVIE"} ):
                            try:
                                shows.append(each["TITLE"])
                                print(shows)
                            except KeyError:
                                continue
                        for each in inp:
                            match = re.search(r'\d\d\d\d', each)
                            if match:
                                key = int(each)

                            for each in netflix.netflix.find( {"type": "MOVIE"} ):
                                show_names.append(each["title"])
                            for each in netflix.netflix.find():
                                try:
                                    if each["TITLE"] in show_names:
                                        shows.append(each)
                                    for each in shows:
                                        if each["RELEASE_YEAR"] == key:
                                            scores.append(each["SCORE"])
                                            options.append(each)
                                    try:
                                        top = max(scores)
                                    except ValueError:
                                        continue
                                    for each in options:
                                        if ("SCORE", top) in each.items() and each["RELEASE_YEAR"] == key:
                                            title = each["TITLE"]
                                except KeyError:
                                    break
                        for each in netflix.netflix.find({"title":title}):
                            id_lab = each["id"]
                        for each in netflix.netflix.find({"id":id_lab}):
                            if ("role","ACTOR") in each.items():
                                actor.append(each["name"])
                        print('The top 5 actors in', title,'are',actor[0]+",",actor[1]+",",actor[2]+",",actor[3]+", and",actor[4]+".")
                
                elif tg["tag"] == "q6": # Top 5 actors in top show in [year]?
                    if "show" in inp and "seasons" not in inp and "season" not in inp and "who" in inp:
                        print('Let me think...')
                        shows = []
                        actor = []
                        show_names = []
                        scores = []
                        options = []

                        for each in netflix.netflix.find( {"type": "SHOW"} ):
                            try:
                                shows.append(each["TITLE"])
                                print(shows)
                            except KeyError:
                                continue
                        for each in inp:
                            match = re.search(r'\d\d\d\d', each)
                            if match:
                                key = int(each)

                            for each in netflix.netflix.find( {"type": "SHOW"} ):
                                show_names.append(each["title"])
                            for each in netflix.netflix.find():
                                try:
                                    if each["TITLE"] in show_names:
                                        shows.append(each)
                                    for each in shows:
                                        if each["RELEASE_YEAR"] == key:
                                            scores.append(each["SCORE"])
                                            options.append(each)
                                    try:
                                        top = max(scores)
                                    except ValueError:
                                        continue
                                    for each in options:
                                        if ("SCORE", top) in each.items() and each["RELEASE_YEAR"] == key:
                                            title = each["TITLE"]
                                except KeyError:
                                    break
                        for each in netflix.netflix.find({"title":title}):
                            id_lab = each["id"]
                        for each in netflix.netflix.find({"id":id_lab}):
                            if ("role","ACTOR") in each.items():
                                actor.append(each["name"])
                        print('The top 5 actors in', title,'are',actor[0]+",",actor[1]+",",actor[2]+",",actor[3]+", and",actor[4]+".")
                elif tg['tag'] == 'q7' and "seasons" in inp: # Number of seasons in top show of [year]?
                    if "show" in inp and "Who" not in inp and "who" not in inp and "seasons" in inp:
                        shows = []
                        for each in netflix.netflix.find( {"type": "SHOW"} ):
                            try:
                                shows.append(each["TITLE"])
                            except KeyError:
                                continue
                        for each in inp:
                            match = re.search(r'\d\d\d\d', each)
                            if match:
                                key = int(each)
                                show_names = []
                                shows = []
                                scores = []
                                options = []
                                for each in netflix.netflix.find( {"type": "SHOW"} ):
                                    show_names.append(each["title"])
                                for each in netflix.netflix.find():
                                    try:
                                        if each["TITLE"] in show_names:
                                            shows.append(each)
                                        for each in shows:
                                            if each["RELEASE_YEAR"] == key:
                                                scores.append(each["SCORE"])
                                            options.append(each)
                                        try:
                                            top = max(scores)
                                        except ValueError:
                                            continue
                                        for each in options:
                                            if ("SCORE", top) in each.items() and each["RELEASE_YEAR"] == key:
                                                title = each["TITLE"]
                                                #print(title)
                                    except KeyError:
                                        break

                                for each in netflix.netflix.find({"TITLE":title}):
                                    num_season = each["NUMBER_OF_SEASONS"]
                                print("There were",num_season,"seasons in",title+".")
                elif tg["tag"] == "q8": # Number of [genre] movies in [year]?
                    try:
                        if "movies" in inp or "movie" in inp and "Who" not in inp and "who" not in inp and "5" not in inp and "five" not in inp and "Five" not in inp:
                            genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                            movie_names = []
                            count = 0
                            for each in inp:
                                if each in genres:
                                    genre = each
                                match = re.search(r'\d\d\d\d', each)
                                if match:
                                    year = int(each)
                            for each in netflix.netflix.find( {"type": "MOVIE"} ):
                                movie_names.append(each["title"])
                            for each in netflix.netflix.find( {"MAIN_GENRE": genre} ):
                                for mov in movie_names:
                                    if ("TITLE", mov) in each.items() and ("RELEASE_YEAR", year) in each.items():
                                        count += 1
                            print(count,"of the best movies on Netflix released in",year,"were in the",genre,"genre.")
                    except UnboundLocalError:
                        continue
                elif tg["tag"] == "q9": # Top 5 movies released in a given year
                    if "movies" in inp and "Who" not in inp and "who" not in inp:
                        genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                        tally = 0
                        genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                        for each in genres:
                            if each in inp:
                                tally +=1
                        if tally == 0:
                            shows = []
                            title = []
                            for each in netflix.netflix.find( {"type": "MOVIE"} ):
                                try:
                                    shows.append(each["TITLE"])
                                except KeyError:
                                    continue
                            for each in inp:
                                match = re.search(r'\d\d\d\d', each)
                                if match:
                                    key = int(each)
                                    show_names = []
                                    shows = []
                                    scores = []
                                    #options = []
                            for each in netflix.netflix.find( {"type": "MOVIE"} ):
                                show_names.append(each["title"])
                            for each in netflix.netflix.find( {"RELEASE_YEAR":key} ):
                                try:
                                    if each["TITLE"] in show_names:
                                        shows.append(each)
                                except KeyError:
                                    break
                            for each in shows:
                                scores.append(each["SCORE"])
                            scores.sort()
                            if len(scores) >= 5:
                                top = scores[0:5]
                            else:
                                top = scores[0:len(scores)]
                            if len(top) == 5:
                                for each in shows:
                                    if ("SCORE", top[0]) in each.items() or ("SCORE", top[1]) in each.items() or ("SCORE", top[2]) in each.items() or ("SCORE", top[3]) in each.items() or ("SCORE", top[4]) in each.items():
                                        title.append(each["TITLE"])

                            else:
                                for each in shows:
                                    for i in range(0,len(top)):
                                        while i <= 5:
                                            if ("SCORE",top[i]) in each.items():
                                                title.append(each["TITLE"])
                            title = np.unique(title)
                            if len(title) < 5:
                                print("There were less than five top movies in",str(key) + ". The top",len(title),"movies in",key,"were:")
                            else:
                                print("The top",len(title),"movies in",key,"were:")
                            for i in range(0,len(title)):
                                print(title[i])

                elif tg["tag"] == "q10": # Top 5 shows released in a given year
                    if "shows" in inp and "Who" not in inp and "who" not in inp:
                        shows = []
                        title = []
                        tally = 0
                        genres = ["documentary", "scifi", "drama", "comedy", "western", "action", "crime", "thriller", "war", "fantasy", "romance", "horror", "musical","animation", "sports"]
                        for each in genres:
                            if each in inp:
                                tally +=1
                        if tally == 0:
                            for each in netflix.netflix.find( {"type": "SHOW"} ):
                                try:
                                    shows.append(each["TITLE"])
                                except KeyError:
                                    continue
                            for each in inp:
                                match = re.search(r'\d\d\d\d', each)
                                if match:
                                    key = int(each)
                                    show_names = []
                                    shows = []
                                    scores = []
                                    #options = []
                                    for each in netflix.netflix.find( {"type": "SHOW"} ):
                                        show_names.append(each["title"])
                                    for each in netflix.netflix.find( {"RELEASE_YEAR":key} ):
                                        try:
                                            if each["TITLE"] in show_names:
                                                shows.append(each)
                                        except KeyError:
                                            break
                                    for each in shows:
                                        scores.append(each["SCORE"])
                                    scores.sort()
                                    if len(scores) >= 5:
                                        top = scores[0:5]
                                    else:
                                        top = scores[0:len(scores)]
                                    if len(top) == 5:
                                        for each in shows:
                                            if ("SCORE", top[0]) in each.items() or ("SCORE", top[1]) in each.items() or ("SCORE", top[2]) in each.items() or ("SCORE", top[3]) in each.items() or ("SCORE", top[4]) in each.items():
                                                title.append(each["TITLE"])

                                    else:
                                        for each in shows:
                                            for i in range(0,len(top)):
                                                while i <= 5:
                                                    if ("SCORE",top[i]) in each.items():
                                                        title.append(each["TITLE"])
                            title = np.unique(title)
                            if len(title) < 5:
                                print("There were less than five top TV shows in",str(key) + ". The top",len(title),"TV shows in",key,"were:")
                            else:
                                print("The top",len(title),"TV shows in",key,"were:")
                            for i in range(0,len(title)):
                                print(title[i])
        else:
            if "help" not in inp:
                print("I didnt get that. Can you explain or try again?")
                print("Here are the types of questions I can answer:\n1. Finding the top show in a given year\n2. Finding the top movie in a given year\n3. Finding the top movie in a given genre\n4. Finding the top show in a given genre\n5. Finding the top 5 actors in the top show in a given year\n6. Finding the top 5 actors in the top movie in a given year\n7. Finding the number of seasons in the top show in a given year\n8. Finding the top 5 shows released in a given year\n9. Finding the top 5 movies released in a given year\n10. Finding the rating of the top movie in a given year")
chat()

Start talking with the bot! (type quit to stop)

Type "help" for information on what I can answer!

When asking questions, specify whether you are asking about movie or tv show data!

For reference, listed movie genres are:
documentary
scifi
drama
comedy
western
action
crime
thriller
war
fantasy
romance
horror
musical
animation
sports
You: help
Here are the types of questions I can answer:
1. Finding the top show in a given year
2. Finding the top movie in a given year
3. Finding the top movie in a given genre
4. Finding the top show in a given genre
5. Finding the top 5 actors in the top show in a given year
6. Finding the top 5 actors in the top movie in a given year
7. Finding the number of seasons in the top show in a given year
8. Finding the top 5 shows released in a given year
9. Finding the top 5 movies released in a given year
10. Finding the rating of the top movie in a given year
You: quit
